<div class="alert alert-block alert-danger">

# FIT5196 Task 2 in Assessment 1
    
#### Student Name: xxxxxxx
#### Student ID: xxxxxxxxx

Date: xxxxxxxx

Environment: xxxxxx

Libraries used:
* os (for interacting with the operating system, included in Python xxxx) 
* pandas 1.1.0 (for dataframe, installed and imported) 
* multiprocessing (for performing processes on multi cores, included in Python 3.6.9 package) 
* itertools (for performing operations on iterables)
* nltk 3.5 (Natural Language Toolkit, installed and imported)
* nltk.tokenize (for tokenization, installed and imported)
* nltk.stem (for stemming the tokens, installed and imported)

    </div>

<div class="alert alert-block alert-info">
    
## Table of Contents

</div>

[1. Introduction](#Intro) <br>
[2. Importing Libraries](#libs) <br>
[3. Examining Input File](#examine) <br>
[4. Loading and Parsing Files](#load) <br>
$\;\;\;\;$[4.1. Tokenization](#tokenize) <br>
$\;\;\;\;$[4.2. Whatever else](#whetev) <br>
$\;\;\;\;$[4.3. Genegrate numerical representation](#whetev1) <br>
[5. Writing Output Files](#write) <br>
$\;\;\;\;$[5.1. Vocabulary List](#write-vocab) <br>
$\;\;\;\;$[5.2. Sparse Matrix](#write-sparseMat) <br>
[6. Summary](#summary) <br>
[7. References](#Ref) <br>

<div class="alert alert-block alert-success">
    
## 1.  Introduction  <a class="anchor" name="Intro"></a>

This assessment concerns textual data and the aim is to extract data, process them, and transform them into a proper format. The dataset provided is in the format of a PDF file containing ....

<div class="alert alert-block alert-success">
    
## 2.  Importing Libraries  <a class="anchor" name="libs"></a>

In this assessment, any python packages is permitted to be used. The following packages were used to accomplish the related tasks:

* **os:** to interact with the operating system, e.g. navigate through folders to read files
* **re:** to define and use regular expressions
* **pandas:** to work with dataframes
* **multiprocessing:** to perform processes on multi cores for fast performance 
* ...

In [18]:
import os
import re
import langid
import pandas as pd
import multiprocessing
from itertools import chain
import nltk
from nltk.probability import *
from nltk.tokenize import RegexpTokenizer
from nltk.tokenize import MWETokenizer
from nltk.stem import PorterStemmer
from nltk.util import ngrams
import json
from nltk.probability import FreqDist
from nltk.collocations import BigramAssocMeasures, BigramCollocationFinder
from sklearn.feature_extraction.text import CountVectorizer
from collections import defaultdict

-------------------------------------

<div class="alert alert-block alert-success">
    
## 3.  Examining Input File <a class="anchor" name="examine"></a>

In [19]:
# from google.colab import drive
# drive.mount('/content/drive')

Let's examine what is the content of the file. For this purpose, .... 

In [22]:
# 打开并加载 JSON 文件
with open('task1_181.json', 'r', encoding='utf-8') as file:
    all_data = json.load(file)

# 初始化一个字典来存储符合条件的 review_text
Temp_review_dictionary = {}

# 遍历所有的数据项
for key, value in all_data.items():
    if isinstance(value, dict):
        reviews = value.get("reviews", [])
        if isinstance(reviews, list):
            business_reviews = []  # 用于存储当前业务的所有 review_text
            for review in reviews:
                # 获取 review_text
                row_review_text = review.get("review_text")
                if row_review_text:  # 确保 review_text 不是 None 或空字符串
                    row_review_text = str(row_review_text)
                    business_reviews.append(row_review_text)
            
            # 如果当前业务的评论数量达到或超过 70 条，则保存该业务的评论
            if len(business_reviews) >= 70:
                Temp_review_dictionary[key] = business_reviews

tokenizer = RegexpTokenizer(r"[a-zA-Z]+")
# 遍历所有的业务评论，并进行词汇分割
for business_id, reviews in Temp_review_dictionary.items():
    tokenized_reviews = []  # 存储每个业务分词后的评论
    for review in reviews:
        # 分词
        tokens = tokenizer.tokenize(review)
        tokenized_reviews.append(tokens)
    
    # 将分词后的评论更新到 review_dictionary 中
    Temp_review_dictionary[business_id] = tokenized_reviews


review_dictionary = {}

# 遍历原始字典
for review_id, word_list in Temp_review_dictionary.items():
    flattened_list = list(chain.from_iterable(word_list))
    review_dictionary[review_id] = flattened_list
review_dictionary
    
with open('stopwords_en.txt', 'r') as f:
    stopwords = set(f.read().splitlines())
# 在字典中去除停用词
for id, flattened_tokens in review_dictionary.items():
    filtered_tokens = []
    for word in flattened_tokens:
        if word not in stopwords:
            filtered_tokens.append(word)
    # 更新字典
    review_dictionary[id] = filtered_tokens


doc_freq = FreqDist()
for wordList in review_dictionary.values():
    unique_words = set(wordList)
    for word in unique_words:
        doc_freq[word] += 1
threshold = 0.95 * len(review_dictionary)
context_dependent_stopwords = [word for word, count in doc_freq.items() if count > threshold]
print(context_dependent_stopwords)
for id, flattened_tokens in review_dictionary.items():
    filtered_tokens = []
    for word in flattened_tokens:
        if word not in context_dependent_stopwords:
            filtered_tokens.append(word)
    # 更新字典
    review_dictionary[id] = filtered_tokens


stemmer = PorterStemmer()
for id, flattened_tokens in review_dictionary.items():
    stemmed_tokens = []
    for word in flattened_tokens:
        stemmed_word = stemmer.stem(word)
        stemmed_tokens.append(stemmed_word)
    # 更新字典
    review_dictionary[id] = stemmed_tokens


['ve', 'time', 'back', 'nice', 'amazing', 'service', 'staff', 'great', 'love', 'friendly', 'place', 'good', 'don', 'excellent']


In [23]:
doc_freq = FreqDist()
for wordList in review_dictionary.values():
    unique_words = set(wordList)
    for word in unique_words:
        doc_freq[word] += 1
threshold = 0.05 * len(review_dictionary)
rare_words = [word for word, count in doc_freq.items() if count < threshold]
print(rare_words)
for id, flattened_tokens in review_dictionary.items():
    filtered_tokens = []
    for word in flattened_tokens:
        if word not in rare_words:
            filtered_tokens.append(word)
    review_dictionary[id] = filtered_tokens

filtered_review_dictionary = {}

for id, flattened_tokens in review_dictionary.items():
    filtered_tokens = [word for word in flattened_tokens if len(word) >= 3]
    filtered_review_dictionary[id] = filtered_tokens

review_dictionary = filtered_review_dictionary



['amus', 'versu', 'joseph', 'heck', 'unsur', 'powerpoint', 'rancheria', 'fasten', 'jt', 'den', 'definet', 'moseley', 'tinker', 'teambuild', 'addict', 'exercis', 'januari', 'boggl', 'old', 'jailbreak', 'sequenc', 'roommat', 'newbi', 'ypu', 'unforeseen', 'het', 'uncov', 'ashley', 'crime', 'prop', 'captur', 'media', 'freebi', 'grandkid', 'brain', 'puzzl', 'countrysid', 'immers', 'script', 'allot', 'ro', 'idiot', 'shasta', 'brightli', 'voupl', 'riddl', 'magnet', 'optim', 'anniversari', 'beep', 'these', 'thrive', 'bond', 'neon', 'teas', 'intric', 'cousin', 'jail', 'prison', 'plot', 'knight', 'impact', 'beginn', 'doabl', 'actor', 'hilltop', 'cultur', 'greedi', 'camper', 'dui', 'vandal', 'employess', 'rv', 'rewir', 'phil', 'akway', 'randi', 'trooper', 'apprehens', 'litter', 'cab', 'goin', 'wholeheartedli', 'isuzu', 'rig', 'satisfactori', 'cirtainli', 'exterior', 'wd', 'charges', 'sixti', 'redicul', 'percent', 'serfvic', 'eureka', 'defentyli', 'spark', 'certif', 'brake', 'emiss', 'carolyn', 'd

It is noteiced that file contains ....

In [24]:
review_dictionary

{'0x54d2ecb7fe7b948f:0x82ba9605c289a0c': ['want',
  'plan',
  'birthday',
  'escap',
  'experi',
  'hope',
  'group',
  'accommod',
  'member',
  'parti',
  'delay',
  'hold',
  'room',
  'fun',
  'make',
  'day',
  'special',
  'daughter',
  'friend',
  'birthday',
  'break',
  'room',
  'absolut',
  'blast',
  'come',
  'room',
  'famili',
  'crazi',
  'amount',
  'fun',
  'find',
  'escap',
  'room',
  'wait',
  'room',
  'hear',
  'room',
  'joe',
  'hospit',
  'assist',
  'updat',
  'complet',
  'room',
  'fun',
  'interact',
  'famili',
  'friend',
  'fun',
  'room',
  'lot',
  'fun',
  'solv',
  'room',
  'ran',
  'room',
  'come',
  'fun',
  'room',
  'red',
  'counti',
  'area',
  'fun',
  'complet',
  'room',
  'lot',
  'fun',
  'room',
  'built',
  'experi',
  'experi',
  'fun',
  'room',
  'owner',
  'price',
  'fun',
  'spend',
  'hour',
  'day',
  'put',
  'think',
  'cap',
  'awesom',
  'super',
  'break',
  'room',
  'wife',
  'coupl',
  'blast',
  'forward',
  'date',


Having parsed the pdf file, the following observations can be made: ....


In [25]:
bigram_measures = nltk.collocations.BigramAssocMeasures()
tokenized_reviews = list(review_dictionary.values())
finder = BigramCollocationFinder.from_documents(tokenized_reviews)
top_200_bigrams = finder.nbest(bigram_measures.pmi, 200)
#reference from chatGPT
collocated_bigrams = [' '.join(bigram) for bigram in top_200_bigrams]

unigrams = set(chain.from_iterable(review_dictionary.values()))
vocab = sorted(list(unigrams) + collocated_bigrams)
with open("181_vocab.txt", "w") as vocab_file:
    for index, word in enumerate(vocab):
        vocab_file.write(f"{word}:{index}\n")

In [27]:
vectorizer = CountVectorizer(vocabulary=vocab)
reviews_as_text = [' '.join(tokens) for tokens in review_dictionary.values()]
X = vectorizer.transform(reviews_as_text)

# 获取每个业务 ID 的顺序列表，以确保 ID 与评论文本一一对应
business_ids = list(review_dictionary.keys())

# 打开文件，写入稀疏矩阵数据
with open(f"181_countvec.txt", "w") as countvec_file:
    for i, business_id in enumerate(business_ids):
        row = X[i].tocoo()  # 获取稀疏矩阵行的 COOrdinate 格式
        tokens_freq = [f"{col}:{val}" for col, val in zip(row.col, row.data)]
        line = f"{business_id}, " + ", ".join(tokens_freq) + "\n"
        countvec_file.write(line)

<div class="alert alert-block alert-success">
    
## 4.  Loading and Parsing File <a class="anchor" name="load"></a>

In this section, ....

Let's examine the dictionary generated. For counting the total number of reviews extracted ....

<div class="alert alert-block alert-warning">
    
### 4.1. Tokenization <a class="anchor" name="tokenize"></a>

Tokenization is a principal step in text processing and producing unigrams. In this section, ....

The above operation results in a dictionary with PID representing keys and a single string for all reviews of the day concatenated to each other. ...

filtered_reviews

At this stage, all reviews for each PID are tokenized and are stored as a value in the new dictionary (separetely for each day).

-------------------------------------

<div class="alert alert-block alert-warning">
    
### 4.2. Whatever else <a class="anchor" name="whetev"></a>

<div class="alert alert-block alert-warning">
    
### 4.3. Generate numerical representation<a class="anchor" name="bigrams"></a>

One of the tasks is to generate the numerical representation for all tokens in abstract.  .....

In [ ]:
#some code

In [ ]:
#some code

Random descriptions and justification ...

In [ ]:
#some code

At this stage, we have a dictionary of tokenized words, whose keys are indicative of....

-------------------------------------

#### Whatever else <a class="anchor" name="whatev1"></a>

<div class="alert alert-block alert-success">
    
## 5. Writing Output Files <a class="anchor" name="write"></a>

files need to be generated:
* Vocabulary list
* Sparse matrix (count_vectors)

This is performed in the following sections.

<div class="alert alert-block alert-warning">
    
### 5.1. Vocabulary List <a class="anchor" name="write-vocab"></a>

List of vocabulary should also be written to a file, sorted alphabetically, with their reference codes in front of them. This file also refers to the sparse matrix in the next file. For this purpose, .....

In [ ]:
#some code

<div class="alert alert-block alert-warning">
    
### 5.2. Sparse Matrix <a class="anchor" name="write-sparseMat"></a>

For writing sparse matrix for a paper, we firstly calculate the frequency of words for that paper ....

In [ ]:
#some code

-------------------------------------

<div class="alert alert-block alert-success">
    
## 6. Summary <a class="anchor" name="summary"></a>

.....

-------------------------------------

<div class="alert alert-block alert-success">
    
## 7. References <a class="anchor" name="Ref"></a>

[1] Pandas dataframe.drop_duplicates(), https://www.geeksforgeeks.org/python-pandas-dataframe-drop_duplicates/, Accessed 13/08/2022.



## --------------------------------------------------------------------------------------------------------------------------